# Test run of Atom functions
This code tests the atom functions provided by Eric Bylaska

In [1]:
from __future__ import print_function

import sys,math
sys.path.insert(0,'C:/Users/pazd068/Documents/Bitbucket/atom_structure/Eric/')
#import fastatomfm as atomfm


#xyzfilename = "small.xyz"
xyzfilename = "C:/Users/pazd068/Documents/Bitbucket/atom_structure/Eric/eatomstructure.xyz"


def isFloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False


def read_in_lines(filename,nlines):
    """Lazy function (generator) to read a file in 
      nline chuncks."""
    with open(filename,'r') as fp:
        while True:
            data = ''.join([fp.readline() for _ in range(nlines)])
            if not data: 
                break
            yield data

def read_xyzfile(filename):
    """Lazy function (generator) to read a xyzfile 
      frame by frame."""
    tobohr = 1.0/0.529177
    print("filename=",filename)
    with open(filename,'r') as ff:
        xyzdata = ff.readline()
    nion = int(xyzdata.split('\n')[0])
    nlines = nion+2
    
    with open(filename,'r') as fp:
        while True:
            data = ''.join([fp.readline() for _ in range(nlines)])
            if not data:
                break
            lines = data.strip().split('\n')
            energy  = float(lines[1])
            symbols = [ln.split()[0] for ln in lines[2:]]
            rions   = [tobohr*float(ln.split()[i]) for ln in lines[2:] for i in range(1,4)]
            fions   = [tobohr*float(ln.split()[i]) for ln in lines[2:] for i in range(4,7)]
            
            
        yield (symbols,rions,fions,energy)

In [7]:
class AtomsFM:

   def __fcut(self,rcut,r):
      ff = 0.0
      if r < rcut:
         ff = 0.5*(math.cos(math.pi*r/rcut)+1.0)
      return ff

   def __f2exp(self,eta,rs,r):
      return math.exp(-eta*(r-rs)*(r-rs))

   def __f2cos(self,kappa,r):
      return math.cos(kappa*r)

   def __init__(self,parameterslist):
      print("hello???")
      self.p1s = []
      self.p2s = []
      self.p3s = []
      self.p4s = []
      self.p5s = []
      self.fmsize = len(parameterslist)
      count = 0
      for parameters in parameterslist:
         if parameters[0] == 1: self.p1s.append([count]+parameters[1:])
         if parameters[0] == 2: self.p2s.append([count]+parameters[1:])
         if parameters[0] == 3: self.p3s.append([count]+parameters[1:])
         if parameters[0] == 4: self.p4s.append([count]+parameters[1:])
         if parameters[0] == 5: self.p5s.append([count]+parameters[1:])
         count += 1
      print("afm init fmsize=",self.fmsize)
      return

   def __call__(self,rion,i):
      fm = [0.0]*self.fmsize
      nion = int(len(rion)/3)
      for j in range(nion):
         if (i!=j):
            xij = rion[3*i]   - rion[3*j]
            yij = rion[3*i+1] - rion[3*j+1]
            zij = rion[3*i+2] - rion[3*j+2]
            rij2 = xij*xij + yij*yij + zij*zij
            rij  = math.sqrt(rij2)
            for p1 in self.p1s:
               fm[p1[0]] += self.__fcut(p1[1],rij)
            for p2 in self.p2s:
               fm[p2[0]] += self.__f2exp(p2[2],p2[3],rij)*self.__fcut(p2[1],rij)
            for p3 in self.p3s:
               fm[p3[0]] += self.__f2cos(p3[2],rij)*self.__fcut(p3[1],rij)

            for k in range(nion):
               if (i!=k):
                  xik = rion[3*i]   - rion[3*k]
                  yik = rion[3*i+1] - rion[3*k+1]
                  zik = rion[3*i+2] - rion[3*k+2]
                  xjk = rion[3*j]   - rion[3*k]
                  yjk = rion[3*j+1] - rion[3*k+1]
                  zjk = rion[3*j+2] - rion[3*k+2]
                  rik2 = xik*xik + yik*yik + zik*zik
                  rjk2 = xjk*xjk + yjk*yjk + zjk*zjk
                  rik  = math.sqrt(rik2)
                  rjk  = math.sqrt(rjk2)
                  ctheta =(xij*xik + yij*yik + zij*zik)/(rij*rik)
                  if (ctheta>1.0):    ctheta = 1.0
                  if (ctheta<(-1.0)): ctheta = -1.0
                  for p4 in self.p4s:
                     tmp0 = 2.0**(1-p4[4])
                     tmp1 = (1.0-p4[3]*ctheta)**p4[4]
                     tmp2 = math.exp(-p4[2]*(rij2 + rik2 + rjk2))
                     fm[p4[0]] += tmp0*tmp1*tmp2*self.__fcut(p4[1],rij)*self.__fcut(p4[1],rik)*self.__fcut(p4[1],rjk)
                  for p5 in self.p5s:
                     tmp0 = 2.0**(1-p5[4])
                     tmp1 = (1.0-p5[3]*ctheta)**p5[4]
                     tmp2 = math.exp(-p5[2]*(rij2 + rik2))
                     fm[p4[0]] += tmp0*tmp1*tmp2*self.__fcut(p5[1],rij)*self.__fcut(p5[1],rik)
      return fm

In [3]:

#### read in parameters ####
parameters = []
with open("parameters",'r') as ff: 
    data = ff.read()
    for line in data.split('\n'):
        ss = line.split()
        if (len(ss)>1):
            p = []
            for s in ss:
                if s.isdigit():  
                    p += [int(s)]
                elif isFloat(s): 
                    p += [float(s)]
            ok = p[0] in range(1,6)
            if (p[0]==1): ok = ok and (len(p)==2)
            if (p[0]==2): ok = ok and (len(p)==4)
            if (p[0]==3): ok = ok and (len(p)==3)
            if (p[0]==4): ok = ok and (len(p)==5)
            if (p[0]==5): ok = ok and (len(p)==5)
            if ok: parameters.append(p)

print("parameters=",parameters,len(parameters))


parameters= [[2, 11.338, 0.001, 0.0], [2, 11.338, 0.01, 0.0], [2, 11.338, 0.02, 0.0], [2, 11.338, 0.035, 0.0], [2, 11.338, 0.06, 0.0], [2, 11.338, 0.1, 0.0], [2, 11.338, 0.2, 0.0], [2, 11.338, 0.4, 0.0], [4, 11.338, 0.0001, -1.0, 1.0], [4, 11.338, 0.0001, 1.0, 1.0], [4, 11.338, 0.0001, -1.0, 2.0], [4, 11.338, 0.0001, 1.0, 2.0], [4, 11.338, 0.003, -1.0, 1.0], [4, 11.338, 0.003, 1.0, 1.0], [4, 11.338, 0.003, -1.0, 2.0], [4, 11.338, 0.003, 1.0, 2.0], [4, 11.338, 0.008, -1.0, 1.0], [4, 11.338, 0.008, 1.0, 1.0], [4, 11.338, 0.008, -1.0, 2.0], [4, 11.338, 0.008, 1.0, 2.0], [4, 11.338, 0.015, -1.0, 1.0], [4, 11.338, 0.015, 1.0, 1.0], [4, 11.338, 0.015, -1.0, 2.0], [4, 11.338, 0.015, 1.0, 2.0], [4, 11.338, 0.015, -1.0, 4.0], [4, 11.338, 0.015, 1.0, 4.0], [4, 11.338, 0.015, -1.0, 16.0], [4, 11.338, 0.015, 1.0, 16.0], [4, 11.338, 0.025, -1.0, 1.0], [4, 11.338, 0.025, 1.0, 1.0], [4, 11.338, 0.025, -1.0, 2.0], [4, 11.338, 0.025, 1.0, 2.0], [4, 11.338, 0.025, -1.0, 4.0], [4, 11.338, 0.025, 1.0, 4.0

In [11]:
#### define Atomic Feature Mapping ####
afm = AtomsFM(parameters)
print(type(afm))


hello???
afm init fmsize= 52
<class '__main__.AtomsFM'>


In [5]:

##### read in atoms lazilly ####
#with open("../eatomstructure.xyz",'r') as ff: 
#   xyzdata = ff.readline()
#nion = int(xyzdata.split('\n')[0])
#print("nion=",nion)

#count = 0
#tobohr = 1.0/0.529177
#for frame in read_in_lines("../eatomstructure.xyz",nion+2):
#   lines = frame.strip().split('\n')
#   energy = float(lines[1])
#   rions  = [tobohr*float(ln.split()[i]) for ln in lines[2:] for i in range(1,4)]
#   fions  = [tobohr*float(ln.split()[i]) for ln in lines[2:] for i in range(4,7)]
#
#   framefm = []
#   for ii in range(nion):
#      framefm.append(afm(rions,ii))
#
#   print("\n")
#   print("frame number,framefm=",count,framefm)
#   print("\n")
#   print("count=",count)
#   print("energy=",energy)
#   print("rions= ",rions)
#   print("fions= ",fions)
#   count += 1

In [12]:

count = 0
for (symbols,rions,fions,energy) in read_xyzfile(xyzfilename):
    nion = len(symbols)
    framefm = []
    for ii in range(nion):
        framefm.append(afm(rions,ii))
        
    print("\n")
    print("frame number,framefm=",count,framefm)
    print("\n")
    print("count=",count)
    print("nion=  ",nion)
    print("energy=",energy)
    print("rions= ",rions)
    print("fions= ",fions)
    count += 1


filename= C:/Users/pazd068/Documents/Bitbucket/atom_structure/Eric/eatomstructure.xyz
14


frame number,framefm= 0 [[2.384985684406592, 1.812291133942458, 1.382924806189201, 0.9625536613639194, 0.5589710749380614, 0.25091364753809786, 0.039946115388959605, 0.001444054535781627, 3.6714916953164063, 0.8162053189170427, 3.4245518033130873, 0.5692654269137234, 3.0321952839816024, 0.5912988261752312, 2.85625314836959, 0.4153566905632193, 2.2433050475359226, 0.34169957045515714, 2.143945545450544, 0.24234006836977728, 1.5408214207335713, 0.16042531945528865, 1.4952450120328014, 0.1148489107545189, 1.4691404678847368, 0.06492968542414167, 1.41231804068008, 0.0024309244627869105, 0.9587207949782109, 0.0553332055362785, 0.943336656619288, 0.03994906717735537, 0.9348495665090236, 0.022750770405733815, 0.916316204884396, 0.0008670446268648207, 0.4166074290909935, 0.006793858571104374, 0.414760718941381, 0.0049471484214918355, 0.4137808374522564, 0.002840635747024378, 0.4116397831937041, 0.0001110